In [ ]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, encoding_dim),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Define Attention Mechanism
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.attn = nn.Linear(hidden_size * 2, hidden_size)
        self.v = nn.Parameter(torch.rand(hidden_size))

    def forward(self, hidden, encoder_outputs):
        seq_len = encoder_outputs.size(1)
        hidden = hidden.unsqueeze(1).repeat(1, seq_len, 1)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))
        energy = energy.transpose(1, 2)
        v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1)
        attention_weights = torch.bmm(v, energy).squeeze(1)
        return torch.softmax(attention_weights, dim=1)

# Define Encoder
class LSTMEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super(LSTMEncoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)

    def forward(self, x):
        outputs, (hidden, cell) = self.lstm(x)
        return outputs, hidden, cell

# Define Decoder with Attention
class LSTMDecoderWithAttention(nn.Module):
    def __init__(self, hidden_size, output_size, num_layers, dropout):
        super(LSTMDecoderWithAttention, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.attention = Attention(hidden_size)
        self.lstm = nn.LSTM(hidden_size + 4, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, hidden, cell, encoder_outputs, current_temporal_features):
        attention_weights = self.attention(hidden[-1], encoder_outputs)
        attention_weights = attention_weights.unsqueeze(1)
        context = torch.bmm(attention_weights, encoder_outputs)
        decoder_input = torch.cat([context, current_temporal_features], dim=2)
        output, (hidden, cell) = self.lstm(decoder_input, (hidden, cell))
        prediction = self.fc(output).squeeze(1)
        return prediction, hidden, cell

# Define full Encoder-Decoder Model
class LSTMEncoderDecoderWithAttention(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, dropout):
        super(LSTMEncoderDecoderWithAttention, self).__init__()
        self.encoder = LSTMEncoder(input_size, hidden_size, num_layers, dropout)
        self.decoder = LSTMDecoderWithAttention(hidden_size, output_size, num_layers, dropout)

    def forward(self, encoder_input, current_temporal_features):
        encoder_outputs, hidden, cell = self.encoder(encoder_input)
        prediction, _, _ = self.decoder(hidden, cell, encoder_outputs, current_temporal_features)
        return prediction

# Load and preprocess data
df = pd.read_csv('time_series_data.csv')

# Define your feature and target columns
features = ['rate_level_1', 'rate_level_2',
            'days_to_end_of_month', 'days_to_ECB_meeting', 'days_to_Fed_meeting', 'ois_sofr_rate']

target = ['rate_level_1', 'rate_level_2']

current_temporal_features = ['days_to_end_of_month', 'days_to_ECB_meeting', 'days_to_Fed_meeting', 'ois_sofr_rate']

# Normalize data
scaler_features = MinMaxScaler()
scaler_target = MinMaxScaler()

df[features] = scaler_features.fit_transform(df[features])
df[target] = scaler_target.fit_transform(df[target])

# Function to create sequences
def create_sequences(data, target_data, n_timesteps):
    X, y, current_time_features = [], [], []
    for i in range(len(data) - n_timesteps):
        X.append(data[i:i + n_timesteps].values)
        y.append(target_data.iloc[i + n_timesteps].values)
        current_time_features.append(data[current_temporal_features].iloc[i + n_timesteps].values)
    return np.array(X), np.array(y), np.array(current_time_features)

# Prepare sequences
n_timesteps_input = 12
X, y, current_temporal = create_sequences(df[features], df[target], n_timesteps_input)

# Split into train and test sets
X_train, X_test, y_train, y_test, current_temporal_train, current_temporal_test = train_test_split(
    X, y, current_temporal, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)
current_temporal_train = torch.tensor(current_temporal_train, dtype=torch.float32)
current_temporal_test = torch.tensor(current_temporal_test, dtype=torch.float32)

# Define and train the autoencoder
autoencoder = Autoencoder(input_dim=len(features) + len(current_temporal_features), encoding_dim=3)
ae_optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)
ae_criterion = nn.MSELoss()

# Combine main features and temporal features for autoencoder training
combined_train = torch.cat((X_train.view(-1, len(features)), current_temporal_train.view(-1, len(current_temporal_features))), dim=1)
combined_test = torch.cat((X_test.view(-1, len(features)), current_temporal_test.view(-1, len(current_temporal_features))), dim=1)

# Train the autoencoder
n_ae_epochs = 50
for epoch in range(n_ae_epochs):
    autoencoder.train()
    ae_optimizer.zero_grad()
    _, decoded = autoencoder(combined_train)
    ae_loss = ae_criterion(decoded, combined_train)
    ae_loss.backward()
    ae_optimizer.step()
    if (epoch + 1) % 10 == 0:
        print(f'Autoencoder Epoch [{epoch+1}/{n_ae_epochs}], Loss: {ae_loss.item():.4f}')

# Transform the features using the trained autoencoder
with torch.no_grad():
    X_train_encoded, _ = autoencoder.encoder(combined_train)
    X_test_encoded, _ = autoencoder.encoder(combined_test)

# Reshape the encoded features back to the original sequence shape
X_train_encoded = X_train_encoded.view(X_train.size(0), n_timesteps_input, -1)
X_test_encoded = X_test_encoded.view(X_test.size(0), n_timesteps_input, -1)

# Hyperparameters
input_size = X_train_encoded.size(2)  # Encoded feature size
hidden_size = 128
output_size = len(target)  # 2 targets
num_layers = 2
dropout = 0.3
learning_rate = 0.001
n_epochs = 50

# Initialize model
model = LSTMEncoderDecoderWithAttention(input_size, hidden_size, output_size, num_layers, dropout)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

# Training loop
for epoch in range(n_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    output = model(X_train_encoded, current_temporal_train.unsqueeze(1))
    
    # Calculate loss
    loss = criterion(output, y_train)
    
    # Backward pass and optimize
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{n_epochs}], Loss: {loss.item():.4f}')

# Example evaluation on test set
model.eval()
with torch.no_grad():
    forecast_encoded = model(X_test_encoded, current_temporal_test.unsqueeze(1))

# Decode the forecasted encoded features back to the original feature space
with torch.no_grad():
    forecast_decoded = autoencoder.decoder(forecast_encoded)

# Inverse-transform the predictions to the original scale
forecast_original_scale = scaler_target.inverse_transform(forecast_decoded.cpu().numpy())
y_test_original_scale = scaler_target.inverse_transform(y_test.cpu().numpy())

# Compare predictions to the actual values
print("Predictions on original scale:", forecast_original_scale)
print("True values on original scale:", y_test_original_scale)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Define Attention Mechanism
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.attn = nn.Linear(hidden_size * 2, hidden_size)
        self.v = nn.Parameter(torch.rand(hidden_size))

    def forward(self, hidden, encoder_outputs):
        seq_len = encoder_outputs.size(1)
        hidden = hidden.unsqueeze(1).repeat(1, seq_len, 1)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))
        energy = energy.transpose(1, 2)
        v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1)
        attention_weights = torch.bmm(v, energy).squeeze(1)
        return torch.softmax(attention_weights, dim=1)

# Define Encoder
class LSTMEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super(LSTMEncoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)

    def forward(self, x):
        outputs, (hidden, cell) = self.lstm(x)
        return outputs, hidden, cell

# Define Decoder with Attention
class LSTMDecoderWithAttention(nn.Module):
    def __init__(self, hidden_size, output_size, num_layers, dropout):
        super(LSTMDecoderWithAttention, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.attention = Attention(hidden_size)
        self.lstm = nn.LSTM(hidden_size + 4, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, hidden, cell, encoder_outputs, current_temporal_features):
        attention_weights = self.attention(hidden[-1], encoder_outputs)
        attention_weights = attention_weights.unsqueeze(1)
        context = torch.bmm(attention_weights, encoder_outputs)
        decoder_input = torch.cat([context, current_temporal_features], dim=2)
        output, (hidden, cell) = self.lstm(decoder_input, (hidden, cell))
        prediction = self.fc(output).squeeze(1)
        return prediction, hidden, cell

# Define full Encoder-Decoder Model
class LSTMEncoderDecoderWithAttention(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, dropout):
        super(LSTMEncoderDecoderWithAttention, self).__init__()
        self.encoder = LSTMEncoder(input_size, hidden_size, num_layers, dropout)
        self.decoder = LSTMDecoderWithAttention(hidden_size, output_size, num_layers, dropout)

    def forward(self, encoder_input, current_temporal_features):
        encoder_outputs, hidden, cell = self.encoder(encoder_input)
        prediction, _, _ = self.decoder(hidden, cell, encoder_outputs, current_temporal_features)
        return prediction

# Load and preprocess data
df = pd.read_csv('time_series_data.csv')

# Define your feature and target columns
features = ['rate_level_1', 'rate_level_2',
            'days_to_end_of_month', 'days_to_ECB_meeting', 'days_to_Fed_meeting', 'ois_sofr_rate']

target = ['rate_level_1', 'rate_level_2']

current_temporal_features = ['days_to_end_of_month', 'days_to_ECB_meeting', 'days_to_Fed_meeting', 'ois_sofr_rate']

# Normalize data
scaler_features = MinMaxScaler()
scaler_target = MinMaxScaler()

df[features] = scaler_features.fit_transform(df[features])
df[target] = scaler_target.fit_transform(df[target])

# Function to create sequences
def create_sequences(data, target_data, n_timesteps):
    X, y, current_time_features = [], [], []
    for i in range(len(data) - n_timesteps):
        X.append(data[i:i + n_timesteps].values)
        y.append(target_data.iloc[i + n_timesteps].values)
        current_time_features.append(data[current_temporal_features].iloc[i + n_timesteps].values)
    return np.array(X), np.array(y), np.array(current_time_features)

# Prepare sequences
n_timesteps_input = 12
X, y, current_temporal = create_sequences(df[features], df[target], n_timesteps_input)

# Split into train and test sets
X_train, X_test, y_train, y_test, current_temporal_train, current_temporal_test = train_test_split(
    X, y, current_temporal, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)
current_temporal_train = torch.tensor(current_temporal_train, dtype=torch.float32)
current_temporal_test = torch.tensor(current_temporal_test, dtype=torch.float32)

# Define and train the autoencoder
autoencoder = Autoencoder(input_dim=len(features) + len(current_temporal_features), encoding_dim=3)
ae_optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)
ae_criterion = nn.MSELoss()

# Combine main features and temporal features for autoencoder training
combined_train = torch.cat((X_train.view(-1, len(features)), current_temporal_train.view(-1, len(current_temporal_features))), dim=1)
combined_test = torch.cat((X_test.view(-1, len(features)), current_temporal_test.view(-1, len(current_temporal_features))), dim=1)

# Train the autoencoder
n_ae_epochs = 50
for epoch in range(n_ae_epochs):
    autoencoder.train()
    ae_optimizer.zero_grad()
    _, decoded = autoencoder(combined_train)
    ae_loss = ae_criterion(decoded, combined_train)
    ae_loss.backward()
    ae_optimizer.step()
    if (epoch + 1) % 10 == 0:
        print(f'Autoencoder Epoch [{epoch+1}/{n_ae_epochs}], Loss: {ae_loss.item():.4f}')

# Transform the features using the trained autoencoder
with torch.no_grad():
    X_train_encoded, _ = autoencoder.encoder(combined_train)
    X_test_encoded, _ = autoencoder.encoder(combined_test)

# Reshape the encoded features back to the original sequence shape
X_train_encoded = X_train_encoded.view(X_train.size(0), n_timesteps_input, -1)
X_test_encoded = X_test_encoded.view(X_test.size(0), n_timesteps_input, -1)

# Hyperparameters
input_size = X_train_encoded.size(2)  # Encoded feature size
hidden_size = 128
output_size = len(target)  # 2 targets
num_layers = 2
dropout = 0.3
learning_rate = 0.001
n_epochs = 50

# Initialize model
model = LSTMEncoderDecoderWithAttention(input_size, hidden_size, output_size, num_layers, dropout)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

# Training loop
for epoch in range(n_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    output = model(X_train_encoded, current_temporal_train.unsqueeze(1))
    
    # Calculate loss
    loss = criterion(output, y_train)
    
    # Backward pass and optimize
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{n_epochs}], Loss: {loss.item():.4f}')

# Example evaluation on test set
model.eval()
with torch.no_grad():
    forecast = model(X_test_encoded, current_temporal_test.unsqueeze(1))

# Inverse-transform the predictions to the original scale
forecast_original_scale = scaler_target.inverse_transform(forecast.cpu().numpy())
y_test_original_scale = scaler_target.inverse_transform(y_test.cpu().numpy())

# Compare predictions to the actual values
print("Predictions on original scale:", forecast_original_scale)
print("True values on original scale:", y_test_original_scale)